# Aphrodisias analysis notebook


In [1]:
from pathlib import Path

import pygeoprocessing
from osgeo import gdal
import statistics
import math
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.windows import from_bounds
from shapely.geometry import Polygon 


### Defining input datasets

In [ ]:
gis_folder = Path(r"C:\Users\lizad\OneDrive\Desktop\Brown\Dissertation\GIS")

target_nodata = -9999.0

dem_raster_path = gis_folder / "DEM_UTM_35N.tif"
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))

slope_raster_path = gis_folder / "slope_35N.tif"
tri_raster_path = gis_folder / "tri_35N.tif"
landcover_raster_path = gis_folder / "landcover_35N.tif"

church_vector_path = gis_folder / "churches_35N.gpkg"
city_center_vector_path = gis_folder / "center_35N.gpkg"

church_raster_path = gis_folder / "churches_35N.tif"
city_center_raster_path = gis_folder / "center_35N.tif"

### Rasterizing Churches to match the DEM

In [3]:
# Create and burn church locations
if not church_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(church_raster_path),
        gdal.GDT_Byte,
        [0],
    )
    pygeoprocessing.rasterize(
    str(church_vector_path),
    str(church_raster_path),
    [1],
    )

### Rasterizing city center to match the DEM

In [ ]:
# Create and burn city center
if not city_center_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(city_center_raster_path),
        gdal.GDT_Byte,
        [0],
    )
    pygeoprocessing.rasterize(
    str(city_center_vector_path),
    str(city_center_raster_path),
        [1],
    )

### Calculate slope and Terrain Ruggedness Index from DEM

In [ ]:
slope_gdal_ds = gdal.DEMProcessing(str(slope_raster_path), str(dem_raster_path), "slope")
del slope_gdal_ds

# pygeoprocessing.calculate_slope((str(dem_raster_path), 1), str(slope_raster_path))

tri_gdal_ds = gdal.DEMProcessing(str(tri_raster_path), str(dem_raster_path), "tri")
del tri_gdal_ds

### Normalize input datasets

In [ ]:
#Align and clip landscape data to DEM extent
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))
dem_raster_clip = gis_folder / "DEM_UTM_35N_clip.tif"
landcover_raster_clip = gis_folder / "landcover_35N_clip.tif"
slope_raster_clip = gis_folder / "slope_35N_clip.tif"
tri_raster_clip = gis_folder / "tri_35N_clip.tif"

pygeoprocessing.align_and_resize_raster_stack(
    [str(dem_raster_path), str(landcover_raster_path), str(slope_raster_path), str(tri_raster_path)],
    [str(dem_raster_clip), str(landcover_raster_clip), str(slope_raster_clip), str(tri_raster_clip)],
    ["near", "near", "near", "near"],
    dem_raster_info["pixel_size"],
    dem_raster_info["bounding_box"],
    raster_align_index=0,
)

### Reclassify landcover into cost surface

In [ ]:
reclass_df = pd.read_csv("./data/esa_worldcover_classification.csv")
cost_value = "cost_value"
landcover_reclass = gis_folder / "landcover_reclass.tif"

reclass_dict = reclass_df.set_index("lucode").to_dict()[cost_value]

pygeoprocessing.reclassify_raster(
    (str(landcover_raster_clip),1),
    reclass_dict,
    str(landcover_reclass),
    gdal.GDT_Float32,
    target_nodata,
)

### Calculate Tobler's hiking function

In [19]:
tobler_surface_raster = gis_folder / "tobler_surface.tif"

slope_raster_info = pygeoprocessing.get_raster_info(str(slope_raster_clip))
slope_cell_resolution = statistics.mean([abs(x) for x in slope_raster_info["pixel_size"]])
slope_nodata = slope_raster_info["nodata"][0]

# def tobler_op(slope_array):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array*math.pi/180)+0.05)))
#     return result
# def tobler_op(slope):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))
#     return result

def tobler_op(slope_array):
    # Make an array of the same shape full of nodata
    output = np.full(slope_array.shape, slope_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(slope_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(slope_array, slope_nodata)

    output[valid_mask] = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array[valid_mask]*math.pi/180)+0.05)))

    return output

if not tobler_surface_raster.exists():
    pygeoprocessing.raster_calculator(
        [(str(slope_raster_clip),1)], 
        tobler_op,
        str(tobler_surface_raster),
        gdal.GDT_Float32,
        target_nodata,
        calc_raster_stats=True
    )


# Define Tobler rescaling function
def tobler_rescale_op(tobler_surface_array, upper_limit=0.1666667, lower_limit=0.0):
    # Make an array of the same shape full of nodata
    output = np.full(tobler_surface_array.shape, target_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(tobler_surface_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(tobler_surface_array, target_nodata)

    # Calculate initial rescaling
    output[valid_mask] = (tobler_surface_array[valid_mask] - lower_limit)/(upper_limit - lower_limit)

    # Force values larger than the upper_limit to equal one
    upper_limit_mask = (tobler_surface_array >= upper_limit) & valid_mask
    output[upper_limit_mask] = 1

    # Force values smaller than the lower_limit to equal zero
    lower_limit_mask = (tobler_surface_array <= lower_limit) & valid_mask
    output[lower_limit_mask] = 0

    return output

tobler_rescale_raster = gis_folder / "tobler_rescale.tif"

# Rescaling Tobler's original function
if tobler_rescale_raster.exists():
    tobler_rescale_raster.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_surface_raster),1)], 
    tobler_rescale_op,
    str(tobler_rescale_raster),
    gdal.GDT_Float32,
    target_nodata,
)

### Create multicriteria cost surface

In [20]:
cost_surface = gis_folder / "cost_surface.tif"

# Get nodata values for toblers and lulc
landcover_raster_nodata = pygeoprocessing.get_raster_info(str(landcover_reclass))["nodata"][0]
tobler_raster_nodata = pygeoprocessing.get_raster_info(str(tobler_rescale_raster))["nodata"][0]


def weighted_average_op(tobler_array, lulc_cost_array, tobler_weight=0.8, lulc_cost_weight=0.2):
    # Make an array of the same shape full of nodata
    output = np.full(tobler_array.shape, target_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(tobler_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(tobler_array, target_nodata)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(lulc_cost_array, target_nodata)

    # Calculate weighted average
    output[valid_mask] = ((tobler_array[valid_mask]*tobler_weight) + (lulc_cost_array[valid_mask]*lulc_cost_weight)) / (tobler_weight+lulc_cost_weight)

    return output 

if cost_surface.exists():
    cost_surface.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_rescale_raster),1),(str(landcover_reclass),1)],
    weighted_average_op,
    str(cost_surface),
    gdal.GDT_Float32,
    target_nodata,
)

### Calculate combined friction surface using raster calculator

In [ ]:
friction_surface = gis_folder / "friction_surface.tif"

file_list = [
    str(friction_surface),
    str(friction_surface_1),
    str(friction_surface_2)
]

def friction_op(slope, vegetation):
    result = slope*0.8 + vegetation*2
    return result

if not friction_surface.exists():
    pygeoprocessing.raster_calculator(
        file_list,
        friction_op,
        str(friction_surface),
        gdal.GDT_Float32,
        -1,
    )

### Create tiled rasters and INI files for wall-to-wall Circuitscape


In [9]:
import os
import configparser
in_path = gis_folder
cost_surface = gis_folder / "cost_surface.tif"

out_path = gis_folder/ "circuitscape"

blank_ini = gis_folder / "circuitscape/blank_circuitscape_ini.ini"
# blank_ini = gis_folder / "circuitscape/blank_circuitscape_manual.ini"

cost_surface_tile_name = "cost_surface_tile_{}_{}.tif"
cost_surface_tile_buffer_name = "cost_surface_tile_buffer_{}_{}.tif"

westline_name = "westline_{}_{}.tif"
eastline_name = "eastline_{}_{}.tif"
northline_name = "northline_{}_{}.tif"
southline_name = "southline_{}_{}.tif"

tile_size_x = 500
tile_size_y = 500

current_value = 10

# Get full cost surface raster information
ds = gdal.Open(gis_folder / "cost_surface.tif")
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize

# for i in range(0, xsize, tile_size_x):
#     for j in range(0, ysize, tile_size_y):

i=0
j=0

tile_folder = out_path / f"{i}_{j}"
tile_folder.mkdir(exist_ok=True)
# Clip cost surface into tiles
tile_raster_path = tile_folder / cost_surface_tile_name.format(i,j)
com_string = f"gdal_translate -of GTIFF -srcwin {i}, {j}, {tile_size_x}, {tile_size_y} {cost_surface} {tile_raster_path}"
os.system(com_string)

# Clip cost surface into buffered tiles
if i<tile_size_x:
    buffer_x_start = 0
    buffer_x_length = 2*tile_size_x if (2*tile_size_x) < (xsize-buffer_x_start) else xsize-buffer_x_start
else:
    buffer_x_start = i-tile_size_x
    buffer_x_length = 3*tile_size_x if (3*tile_size_x) < (xsize-buffer_x_start) else xsize-buffer_x_start

if j<tile_size_y:
    buffer_y_start = 0
    buffer_y_length = 2*tile_size_y if (2*tile_size_y) < (ysize-buffer_y_start) else ysize-buffer_y_start 
else:
    buffer_y_start = j-tile_size_y
    buffer_y_length = 3*tile_size_y if (3*tile_size_y) < (ysize-buffer_y_start) else ysize-buffer_y_start

buffered_tile_raster_path = tile_folder / cost_surface_tile_buffer_name.format(i,j)
buffer_com_string = f"gdal_translate -of GTIFF -srcwin {buffer_x_start}, {buffer_y_start}, {buffer_x_length}, {buffer_y_length} {cost_surface} {buffered_tile_raster_path}"
os.system(buffer_com_string)

# Create borders around buffered tiles
tile_raster_info = pygeoprocessing.get_raster_info(str(buffered_tile_raster_path))
base_args = {
    "base_array": "",
    "target_nodata": tile_raster_info["nodata"][0],
    "pixel_size":tile_raster_info["pixel_size"],
    "origin": (
        tile_raster_info["bounding_box"][0],
        tile_raster_info["bounding_box"][3],
    ),
    "projection_wkt": tile_raster_info["projection_wkt"],
    "target_path": "",
}

tile_array = pygeoprocessing.raster_to_numpy_array(str(buffered_tile_raster_path))

tile_array.fill(tile_raster_info["nodata"][0])

# Westline
westline_raster = tile_folder / westline_name.format(i,j)
line_array = tile_array.copy()
line_array[:, 0] = current_value
base_args.update(
        {"base_array": line_array, "target_path": str(westline_raster)}
    )
pygeoprocessing.numpy_array_to_raster(
    **base_args,
)

# Eastline
eastline_raster = tile_folder / eastline_name.format(i,j)
line_array = tile_array.copy()
line_array[:, -1] = current_value
base_args.update(
        {"base_array": line_array, "target_path": str(eastline_raster)}
    )
pygeoprocessing.numpy_array_to_raster(
    **base_args,
)

# Northline
northline_raster = tile_folder / northline_name.format(i,j)
line_array = tile_array.copy()
line_array[0] = current_value
base_args.update(
        {"base_array": line_array, "target_path": str(northline_raster)}
    )
pygeoprocessing.numpy_array_to_raster(
    **base_args,
)

# Southline
southline_raster = tile_folder / southline_name.format(i,j)
line_array = tile_array.copy()
line_array[-1] = current_value
base_args.update(
        {"base_array": line_array, "target_path": str(southline_raster)}
    )
pygeoprocessing.numpy_array_to_raster(
    **base_args,
)

# # Iterate through each directional run and run circuitscape
# for run_name, source_raster, ground_raster in zip(
#     ["we", "ew","ns","sn",],
#     [westline_raster, eastline_raster, northline_raster, southline_raster],
#     [eastline_raster, westline_raster, southline_raster, northline_raster,]
# ):

run_name, source_raster, ground_raster = "we", westline_raster, eastline_raster


# Replace habitat, current, and ground rasters in new ini file with buffered tile, current, and ground rasters, and change output locations

run_ini_file = tile_folder / f"{run_name}.ini"

config = configparser.ConfigParser()
config.read(blank_ini)

config.set(
    section="Habitat raster or graph",
    option="habitat_file",
    value=str(buffered_tile_raster_path).replace('\\',r"/")
    )

config.set(
    section="Options for advanced mode",
    option="source_file",
    value=str(source_raster).replace('\\',r"/")
    )

config.set(
    section="Options for advanced mode",
    option="ground_file",
    value=str(ground_raster).replace('\\',r"/")
    )

config.set(
    section="Output options",
    option="output_file",
    value=str(run_ini_file.with_suffix('.out')).replace('\\',r"/")
    )

# config.set(
#     section="Logging Options",
#     option="log_file",
#     value=str(run_ini_file.with_suffix('.log')).replace('\\',r"/")
#     )

# config.set(
#     section="Logging Options",
#     option="profiler_log_file",
#     value=str(run_ini_file.with_name(f'{run_name}_profiler_log_file.out')).replace('\\',r"/")
#     )

# # Create new .ini file in tile folder
with open(run_ini_file, "w") as configfile:
    config.write(configfile)



### Process Circuitscape outputs into final raster

In [ ]:
# TODO Clip outputs by the original (unbuffered) tile

# TODO(OUT OF DIRECTION ITERATOR) Merge all directional outputs into average 'omnidirectional' raster

# TODO(OUT OF TIME ITERATOR) Mosaic all omnidirectional tile outputs into a single raster
